In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33708
2,Huelva,Confirmados PDIA 14 días,844
3,Huelva,Tasa PDIA 14 días,"164,46791511584854"
4,Huelva,Confirmados PDIA 7 días,378
5,Huelva,Total Confirmados,33893
6,Huelva,Curados,32095
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33708.0
/tmp/ipykernel_4221/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9229.0
/tmp/ipykernel_4221/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4221/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4221/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4221/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1296 personas en los últimos 7 días 

Un positivo PCR cada 558 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33708.0,378.0,844.0,513170.0,73.659801,164.467915,159.0
Huelva-Costa,19453.0,259.0,519.0,289548.0,89.449763,179.244892,113.0
Huelva (capital),9229.0,111.0,258.0,143837.0,77.170686,179.369703,47.0
Condado-Campiña,10943.0,98.0,272.0,156231.0,62.727628,174.101171,39.0
Isla Cristina,2223.0,32.0,48.0,21393.0,149.581639,224.372458,20.0
Lepe,2201.0,25.0,40.0,27880.0,89.670014,143.472023,14.0
Almonte,1930.0,12.0,29.0,24507.0,48.965602,118.333537,9.0
Ayamonte,1378.0,31.0,45.0,21104.0,146.891585,213.229719,8.0
Moguer,1431.0,22.0,64.0,21867.0,100.608222,292.678465,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Santa Bárbara de Casa,21.0,5.0,5.0,1043.0,479.386385,479.386385,NaN
Alosno,196.0,14.0,17.0,3933.0,355.962370,432.240020,7.0
Cerro de Andévalo (El),157.0,1.0,9.0,2327.0,42.973786,386.764074,1.0
San Juan del Puerto,595.0,6.0,33.0,9411.0,63.755180,350.653491,2.0
Paymogo,79.0,3.0,4.0,1162.0,258.175559,344.234079,0.0
Bonares,309.0,9.0,18.0,6060.0,148.514851,297.029703,2.0
Moguer,1431.0,22.0,64.0,21867.0,100.608222,292.678465,7.0
Palos de la Frontera,736.0,6.0,29.0,11742.0,51.098620,246.976665,2.0
Manzanilla,109.0,2.0,5.0,2118.0,94.428706,236.071766,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cerro de Andévalo (El),157.0,1.0,9.0,2327.0,42.973786,386.764074,1.0,0.111111
Beas,233.0,1.0,7.0,4341.0,23.036167,161.253167,0.0,0.142857
San Juan del Puerto,595.0,6.0,33.0,9411.0,63.755180,350.653491,2.0,0.181818
Trigueros,335.0,1.0,5.0,7862.0,12.719410,63.597049,1.0,0.200000
Palos de la Frontera,736.0,6.0,29.0,11742.0,51.098620,246.976665,2.0,0.206897
Rociana del Condado,578.0,2.0,8.0,7939.0,25.192090,100.768359,0.0,0.250000
Cartaya,1421.0,13.0,46.0,20083.0,64.731365,229.049445,6.0,0.282609
Palma del Condado (La),1202.0,7.0,22.0,10801.0,64.808814,203.684844,3.0,0.318182
Punta Umbría,863.0,7.0,21.0,15355.0,45.587756,136.763269,4.0,0.333333
